In [1]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np

from faker import Faker
import random
#from tqdm import tqdm
#from babel.dates import format_date
from nn_utils_v2 import *
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [20]:
#loading dataset
m = 1000
m1=8000
dataset, ocr_vocab, machine_vocab, inv_machine_vocab = load_dataset(m)
#first 10 dataset
dataset[:10]
#dataset: a list of tuples of (human readable date, machine readable date)
#ocr_vocab:a python dictionary mapping all characters used in the human readable dates to an integer-valued index
#machine_vocab: a python dictionary mapping all characters used in output invoices to an integer-valued index.
#inv_machine_vocab: the inverse dictionary of machine_vocab, mapping from indices back to characters.
print(dataset[:10])
print(ocr_vocab)
print(machine_vocab)

[('invoice number: 8318-5719-3160', 'Invoice Number: 8318-5719-3160'), ('invoice number: 3882-9337-2635', 'Invoice Number: 3882-9337-2635'), ('invoice number: 5449-2944-5001', 'Invoice Number: 5449-2944-5001'), ('invoice number: 8901-9189-7575', 'Invoice Number: 8901-9189-7575'), ('invoice number: 8260-9493-2809', 'Invoice Number: 8260-9493-2809'), ('invoice number: 8475-6701-8487', 'Invoice Number: 8475-6701-8487'), ('invoice number: 7570-8479-8155', 'Invoice Number: 7570-8479-8155'), ('invoice number: 8045-4813-1817', 'Invoice Number: 8045-4813-1817'), ('invoice number: 6743-5400-3771', 'Invoice Number: 6743-5400-3771'), ('invoice number: 8132-2384-2062', 'Invoice Number: 8132-2384-2062')]
{' ': 0, '#': 1, '-': 2, '.': 3, '0': 4, '1': 5, '2': 6, '3': 7, '4': 8, '5': 9, '6': 10, '7': 11, '8': 12, '9': 13, ':': 14, 'I': 15, 'N': 16, 'a': 17, 'b': 18, 'c': 19, 'd': 20, 'e': 21, 'f': 22, 'g': 23, 'h': 24, 'i': 25, 'k': 26, 'm': 27, 'n': 28, 'o': 29, 'r': 30, 'u': 31, 'v': 32, '<unk>': 33

In [3]:
Tx = 31
Ty = 30
X, Y, Xoh, Yoh = preprocess_data(dataset, ocr_vocab, machine_vocab, Tx,Ty)

In [4]:
print("X.shape:", X.shape)
print("Y.shape:", Y.shape)
print("Xoh.shape:", Xoh.shape)
print("Yoh.shape:", Yoh.shape)

X.shape: (8000, 31)
Y.shape: (8000, 30)
Xoh.shape: (8000, 31, 35)
Yoh.shape: (8000, 30, 26)


In [5]:
index = 6542
print("Source data:", dataset[index][0])
print("Target data:", dataset[index][1])
print()
print("Source after preprocessing (indices):", X[index])
print("Target after preprocessing (indices):", Y[index])
print()
print("Source after preprocessing (one-hot):", Xoh[index])
print("Target after preprocessing (one-hot):", Yoh[index])

Source data: inv no: 1565582
Target data: Invoice Number: 1565582

Source after preprocessing (indices): [25 28 32  0 28 29 14  0  5  9 10  9  9 12  6 34 34 34 34 34 34 34 34 34
 34 34 34 34 34 34 34]
Target after preprocessing (indices): [13 20 24 21 18 16 17  0 14 23 19 15 17 22 12  0  3  7  8  7  7 10  4 25
 25 25 25 25 25 25]

Source after preprocessing (one-hot): [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
Target after preprocessing (one-hot): [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 

In [6]:
# Defined shared layers as global variables
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)

In [7]:
def one_step_attention(a, s_prev):
    
    
    s_prev = repeator(s_prev)
    concat = concatenator([a, s_prev])
    e = densor1(concat)
    energies = densor2(e)
    alphas = activator(energies)
    context = dotor([alphas,a]) 
    
    return context

In [8]:
n_a = 32
n_s = 64
post_activation_LSTM_cell = LSTM(n_s, return_state = True)
output_layer = Dense(len(machine_vocab), activation=softmax)

In [9]:
n_a = 32
n_s = 64
post_activation_LSTM_cell = LSTM(n_s, return_state = True)
output_layer = Dense(len(machine_vocab), activation=softmax)

In [10]:
def model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    
    

    X = Input(shape=(Tx, human_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
    
    
    a = Bidirectional(LSTM(n_a, return_sequences=True),input_shape=(m, Tx, n_a*2))(X)
    
    
    for t in range(Ty):
    
        context = one_step_attention(a, s)
        
        s, _, c = post_activation_LSTM_cell(context,initial_state = [s, c] )
        
        out = output_layer(s)
        
        outputs.append(out)
    
    model = Model(inputs=[X,s0,c0],outputs=outputs)
    
    
    
    return model

In [11]:
#model = model(Tx, Ty, n_a, n_s, len(ocr_vocab), len(machine_vocab))
model = model(Tx, Ty, n_a, n_s, len(ocr_vocab), len(machine_vocab))

In [12]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 31, 35)       0                                            
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 64)           0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 31, 64)       17408       input_1[0][0]                    
__________________________________________________________________________________________________
repeat_vector_1 (RepeatVector)  (None, 31, 64)       0           s0[0][0]                         
                                                                 lstm_2[0][0]                     
          

In [13]:
opt = Adam(lr=0.005, beta_1=0.9, beta_2=0.999,decay=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])

In [14]:
s0 = np.zeros((m1, n_s))
c0 = np.zeros((m1, n_s))
outputs = outputs = list(Yoh.swapaxes(0,1))

In [ ]:
model.fit([Xoh, s0, c0], outputs, epochs=100, batch_size=100)

Epoch 1/100
8000/8000 [==============================] - 18s 2ms/step - loss: 26.1401 - dense_4_loss: 0.5908 - dense_4_acc: 1.0000 - dense_4_acc_1: 1.0000 - dense_4_acc_2: 1.0000 - dense_4_acc_3: 1.0000 - dense_4_acc_4: 1.0000 - dense_4_acc_5: 1.0000 - dense_4_acc_6: 1.0000 - dense_4_acc_7: 1.0000 - dense_4_acc_8: 1.0000 - dense_4_acc_9: 1.0000 - dense_4_acc_10: 1.0000 - dense_4_acc_11: 1.0000 - dense_4_acc_12: 1.0000 - dense_4_acc_13: 1.0000 - dense_4_acc_14: 1.0000 - dense_4_acc_15: 0.9999 - dense_4_acc_16: 0.2584 - dense_4_acc_17: 0.3705 - dense_4_acc_18: 0.1796 - dense_4_acc_19: 0.1779 - dense_4_acc_20: 0.4255 - dense_4_acc_21: 0.4410 - dense_4_acc_22: 0.4412 - dense_4_acc_23: 0.7050 - dense_4_acc_24: 0.7285 - dense_4_acc_25: 0.9999 - dense_4_acc_26: 0.7509 - dense_4_acc_27: 0.7781 - dense_4_acc_28: 0.7795 - dense_4_acc_29: 0.7779

In [39]:
from numpy import newaxis
EXAMPLES = ['invoice ahdoug: 71403', 'invoic no.: 54556', 'invoice no.: 45421', 'invoice number: 70464', 'invoice number: 1858-5672-3871']
for example in EXAMPLES:
    
    source = string_to_int(example, Tx, ocr_vocab,"x")
    #print(source)
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(ocr_vocab)), source)))
    source=source[newaxis,:,:]
    #print(source.shape)
    prediction = model.predict([source, s0, c0])
    prediction = np.argmax(prediction, axis = -1)
    output = [inv_machine_vocab[int(i)] for i in prediction]
    output = [x for x in output if x is not None]
    
    print("source:", example)
    print("output:", "".join(output))

source: invoice ahdoug: 71403
output: Invoice Number: 33004
source: invoic no.: 54556
output: Invoice Number: 55555
source: invoice no.: 45421
output: Invoice Number: 44444
source: invoice number: 70464
output: Invoice Number: 44444
source: invoice number: 1858-5672-3871
output: Invoice Number: 1185-8877-7727
